<a href="https://colab.research.google.com/github/uPistola/uPistola.github.io/blob/main/C%C3%B3pia_de_Untitled13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Seu trabalho será criar um aplicativo usando como base a Linguagem Python para ser executada no
Google Colaboratory. Este aplicativo deve receber Fórmulas escritas em Lógica de Primeiro Grau
usando Latex, validar estas expressões e produzir as fórmulas equivalentes escritos segundo as
formas normais conjuntiva e disjuntiva.
As fórmulas que serão processadas serão lidas de um arquivo de textos contendo, no mínimo, 5
fórmulas bem formadas. Nestas cinco fórmulas devem estar representados todos os operadores e
quantificadores da Lógica de Primeira Ordem.
Todas as impressões, devem ser feitas em Latex. Além disso, todas as fórmulas bem formadas,
estejam na forma normal, ou não, devem estar em notação matemática em todas as telas de saída.

Henrique Zan Grande

João Gabriel Pitol

Lucas Braga Schuck

Rafael Vargas

In [ ]:
import re
import matplotlib.pyplot as plt
from IPython.display import display, Math
import itertools

In [ ]:
# Ler as fórmulas de um arquivo
def ler_formulas_do_arquivo(arquivo):
    with open(arquivo, 'r') as f:
        formulas = f.readlines()
    return [formula.strip() for formula in formulas]

arquivo = ler_formulas_do_arquivo("arquivo.txt")


In [ ]:
latex_to_symbol = {
    r'\wedge': '∧',
    r'\vee': '∨',
    r'\neg': '¬',
    r'\rightarrow': '→',
    r'\leftrightarrow': '↔',
    r'\forall': '∀',
    r'\exists': '∃',
}

def converter_para_latex(texto):
    for latex, symbol in latex_to_symbol.items():
        texto = texto.replace(latex, symbol)
    return texto
formulas_convertidas=[]
for formulas in arquivo:
  formula_latex = converter_para_latex(formulas)
  formulas_convertidas.append(formula_latex)

In [ ]:
def mostrar_formula(lista_formulas):
  for formula in lista_formulas:
    formula_latex = converter_para_latex(formula)
    display(Math(formula_latex))

In [ ]:
def validar_formulas_bem_formadas(formulas):
    for formula in formulas:
        if not validar_sintaxe(formula):
            return False
    return True

def validar_sintaxe(formula):
    # Verifica correspondência de parênteses
    if formula.count('(') != formula.count(')'):
        return False

    return True

if validar_formulas_bem_formadas(formulas_convertidas):
    print("Todas as fórmulas estão bem formadas.")
else:
    print("Alguma das fórmulas não está bem formada.")

Todas as fórmulas estão bem formadas.


In [ ]:
def separar_quantificadores_e_restante(formula):
    quantificadores = ""
    resto = formula

    match = re.match(r'^(∀\w+|∃\w+)+', formula)
    if match:
        quantificadores = match.group(0)
        resto = formula[len(quantificadores):].strip()

    return quantificadores, resto

def combinar_quantificadores(quantificadores, formula):
    return quantificadores+ ":"+ formula


In [ ]:
def eliminar_implicacoes(formula):
    index = formula.find('→')
    while index != -1:
        # Encontre o antecedente (parte esquerda da implicação)
        antecedente_inicio = index - 1
        abre_parenteses = fecha_parenteses = 0

        quantificadores = ['∀', '∃']
        has_quantifier = any(q in formula[:index] for q in quantificadores)

        while antecedente_inicio >= 0 and (formula[antecedente_inicio] not in '∧∨¬(' or abre_parenteses != fecha_parenteses):
            if formula[antecedente_inicio] == '(':
                fecha_parenteses += 1
            elif formula[antecedente_inicio] == ')':
                abre_parenteses += 1
            antecedente_inicio -= 1

        antecedente_inicio += 1

        # Encontre o consequente (parte direita da implicação)
        consequente_fim = index + 2
        abre_parenteses = fecha_parenteses = 0

        while consequente_fim < len(formula) and (formula[consequente_fim] not in '∧∨¬()' or abre_parenteses != fecha_parenteses):
            if formula[consequente_fim] == '(':
                fecha_parenteses += 1
            elif formula[consequente_fim] == ')':
                abre_parenteses += 1
            consequente_fim += 1

        # Substitua a implicação pelo equivalente
        antecedente = formula[antecedente_inicio:index].strip()
        consequente = formula[index + 2:consequente_fim].strip()
        formula = formula[:antecedente_inicio] + f"¬{antecedente} ∨ {consequente}" + formula[consequente_fim:]

        index = formula.find('→')

    return formula



In [ ]:
def aplicar_de_morgan(formula):
    while re.search(r'¬\(([^)]+)∧([^)]+)\)', formula):
        formula = re.sub(r'¬\(([^)]+)∧([^)]+)\)', r'¬ \1∨ ¬\2', formula)

    while re.search(r'¬\(([^)]+)∨([^)]+)\)', formula):
        formula = re.sub(r'¬\(([^)]+)∨([^)]+)\)', r'¬ \1∧ ¬\2', formula)

    return formula



In [ ]:
def eliminar_bi_implicacoes(formula):
    # Combinando todas as possibilidades para a bi-condicional.
    formula = re.sub(r'(\w+)\s*↔\s*(\w+)',r'(\1 → \2) ∧ (\2 → \1)', formula)
    formula = re.sub(r'(\w+)\s*↔\s*\(([^)]+)\)', r'(\1 → \2) ∧ ((\2) → (\1))', formula)
    formula = re.sub(r'\(([^)]+)\)\s*↔\s*(\w+)', r'((\1) → (\2)) ∧ (\2 → \1)', formula)
    formula = re.sub(r'\(([^)]+)\)\s*↔\s*\(([^)]+)\)', r'¬ (\1) ∨ (\2)', formula)
    formula = re.sub(r'(\w+)\s*↔\s*(\w+)', r'(\1 → \2) ∧ (\2 → \1)', formula)


    return formula


In [ ]:
def eliminar_negacoes(formula):
    # Eliminando a dupla negação
    while re.search(r'¬\s*¬', formula):
        formula = re.sub(r'¬\s*¬\s*([A-Za-z][A-Za-z0-9]*)', r'\1', formula)
        formula = re.sub(r'¬\s*¬\s*\(([^)]+)\)', r'\1', formula)
    return formula

In [ ]:
def distribuir(formula):
    changed = True
    while changed:
        changed = False

        # Procura por padrões específicos que podem ser distribuídos
        patterns = [
            (r'\(([^)]+)\)\s*∧\s*\(([^)]+)\)\s*∧\s*(\w+)', r'(\1 ∧ \3) ∧ (\2 ∧ \3)'),
            (r'(\w+)\s*∧\s*\(([^)]+)\)\s*∧\s*\(([^)]+)\)', r'(\1 ∧ \2) ∧ (\1 ∧ \3)'),
            (r'\(([^)]+)\)\s*∧\s*(\w+)\s*∧\s*\(([^)]+)\)', r'(\1 ∧ \3) ∧ (\2 ∧ \3)'),
            (r'(\w+)\s*∧\s*(\w+)\s*∧\s*\(([^)]+)\)', r'(\1 ∧ \3) ∧ (\2 ∧ \3)'),
            (r'\(([^)]+)\)\s*∨\s*\(([^)]+)\)\s*∨\s*(\w+)', r'(\1 ∨ \3) ∨ (\2 ∨ \3)'),
            (r'(\w+)\s*∨\s*\(([^)]+)\)\s*∨\s*\(([^)]+)\)', r'(\1 ∨ \2) ∨ (\1 ∨ \3)'),
            (r'\(([^)]+)\)\s*∨\s*(\w+)\s*∨\s*\(([^)]+)\)', r'(\1 ∨ \3) ∨ (\2 ∨ \3)'),
            (r'(\w+)\s*∨\s*(\w+)\s*∨\s*\(([^)]+)\)', r'(\1 ∨ \3) ∨ (\2 ∨ \3)')
        ]

        for pattern, replacement in patterns:
            new_formula = re.sub(pattern, replacement, formula)
            if new_formula != formula:
                formula = new_formula
                changed = True
                break

    return formula

In [ ]:
def distribuir_para_fnc(formula) :
    changed = True
    while changed :
        changed = False

        # Distribuir ∨ sobre ∧ para obter FNC
        patterns = [
            (r'\((\w)\s*∨\s*\((\w)\s*∧\s*(\w)\)\)', r'((\1 ∨ \2) ∧ (\1 ∨ \3))'),
                (r'\((\w)\s*∧\s*(\w)\)\s*∨\s*(\w)', r'((\1 ∨ \3) ∧ (\2 ∨ \3))'),
                (r'(\w)\s*∨\s*\((\w)\s*∧\s*(\w)\)', r'((\1 ∨ \2) ∧ (\1 ∨ \3))')
        ]

        for pattern, replacement in patterns:
            new_formula = re.sub(pattern, replacement, formula)
            if new_formula != formula:
                formula = new_formula
                changed = True
                break

    return formula

def distribuir_para_fnd(formula):
    changed = True
    while changed:
        changed = False



        # Distribuir ∧ sobre ∨ para obter FND
        patterns = [
            (r'\((\w)\s*∧\s*\((\w)\s*∨\s*(\w)\)\)', r'((\1 ∧ \2) ∨ (\1 ∧ \3))'),
                (r'\((\w)\s*∨\s*(\w)\)\s*∧\s*(\w)', r'((\1 ∧ \3) ∨ (\2 ∧ \3))'),
                (r'(\w)\s*∧\s*\((\w)\s*∨\s*(\w)\)', r'((\1 ∧ \2) ∨ (\1 ∧ \3))')
        ]

        for pattern, replacement in patterns:
            new_formula = re.sub(pattern, replacement, formula)
            if new_formula != formula:
                formula = new_formula
                changed = True
                break

    return formula

def forma_normal_conjuntiva(formula):
    formula = forma_normal(formula)
    formula = distribuir_para_fnc(formula)
    return formula

def forma_normal_disjuntiva(formula):
    formula = forma_normal(formula)
    formula = distribuir_para_fnd(formula)
    return formula

In [ ]:
def forma_normal(formula):

    # Eliminar bi-implicações
    formula = eliminar_bi_implicacoes(formula)


    # Eliminar implicações
    formula = eliminar_implicacoes(formula)

    # Aplicar leis de De Morgan
    formula = aplicar_de_morgan(formula)


    # Eliminar dupla negação
    formula = eliminar_negacoes(formula)
    return formula

final=[]
for formula in formulas_convertidas:
  final.append(forma_normal(formula))
mostrar_formula(final)



<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>